In [51]:
# Criando navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [52]:
# Entrando no navegador
import os

caminho = os.getcwd()
arquivo = caminho + r"\index.html"
# navegador.get(arquivo)

In [53]:
# Importando a base de dados
import pandas as pd

tabela = pd.read_excel("Processos.xlsx")
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Samuel,Samuel adv,PC6592,Distrito Federal,NaN
1,João,João adv,EB3792,Rio de Janeiro,NaN
2,Eliel,Eliel adv,MM1043,Rio de Janeiro,NaN
3,Gabriel,Gabriel adv,PC5197,São Paulo,NaN
4,Reinaldo,Reinaldo adv,PC3332,São Paulo,NaN
5,Raphael,Raphael avc,LS2562,São Paulo,NaN
6,Lucas,Lucas adv,CACA015,São Paulo,NaN


In [54]:
# Iterando na Base de Dados

for linha in tabela.index:
    # para cada processo (linha da tabela)
    navegador.get(arquivo)
    
    # Abrir a lista de cidades
    escolhaEstado = navegador.find_element('xpath', '/html/body/div/div/button')
    ActionChains(navegador).move_to_element(escolhaEstado).perform()

    # Selecionando cidades
    cidade = tabela.loc[linha, 'Cidade']
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

    # Indo para outra aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]
    navegador.switch_to.window(nova_aba)

    #Preenchendo os campos
    
    navegador.find_element('name', 'nome').send_keys(tabela.loc[linha, 'Nome'])
    navegador.find_element('name', 'advogado').send_keys(tabela.loc[linha, 'Advogado'])
    navegador.find_element('name', 'numero').send_keys(tabela.loc[linha, 'Processo'])

    #clicando no botao pesquisar
    navegador.find_element('xpath', '//*[@id="formulario"]/div/button').click()
    
    # selecionar um alerta
    alerta = navegador.switch_to.alert
    alerta.accept()

    # esperar resultado da pesquisa
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)

    texto = alerta.text
    if "Processo encontrado com sucesso" in texto:
        alerta.accept()
        tabela.loc[linha, "Status"] = "Encontrado" 
    else:
        tabela.loc[linha, "Status"] = "Não encontrado" 
        alerta.accept()        
    


C:\Users\samue\AppData\Local\Temp\ipykernel_7788\2236083903.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tabela.loc[linha, "Status"] = "Encontrado"


In [55]:
navegador.quit()
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Samuel,Samuel adv,PC6592,Distrito Federal,Encontrado
1,João,João adv,EB3792,Rio de Janeiro,Encontrado
2,Eliel,Eliel adv,MM1043,Rio de Janeiro,Encontrado
3,Gabriel,Gabriel adv,PC5197,São Paulo,Encontrado
4,Reinaldo,Reinaldo adv,PC3332,São Paulo,Não encontrado
5,Raphael,Raphael avc,LS2562,São Paulo,Não encontrado
6,Lucas,Lucas adv,CACA015,São Paulo,Encontrado
